Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappop, heappush
from collections import defaultdict

In [ ]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

# Stato finale desiderato
GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

In [3]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

# A* with Manhattan Distance

In [4]:
def manhattan_distance(state):
    """Calcola la Manhattan Distance tra lo stato corrente e lo stato obiettivo."""
    total_distance = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            value = state[i, j]
            if value != 0:  # Non calcolare per il blocco vuoto
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                total_distance += abs(goal_x - i) + abs(goal_y - j)
    return total_distance

def a_star(initial_state):
    """Risolvi il puzzle usando l'algoritmo A* con Manhattan Distance."""
    # Coda prioritaria per la frontiera
    frontier = []
    heappush(frontier, (0, initial_state.tobytes()))  # (priorità, stato)

    # Distanze conosciute (g-cost)
    g_cost = {initial_state.tobytes(): 0}

    # Percorso per ricostruire la soluzione
    came_from = {}

    while frontier:
        _, current_bytes = heappop(frontier)
        current_state = np.frombuffer(current_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))

        # Controlla se abbiamo raggiunto l'obiettivo
        if np.array_equal(current_state, GOAL_STATE):
            return reconstruct_path(came_from, current_bytes)

        # Esplora gli stati vicini
        for action in available_actions(current_state):
            neighbor_state = do_action(current_state, action)
            neighbor_bytes = neighbor_state.tobytes()

            tentative_g_cost = g_cost[current_bytes] + 1

            # Se il nuovo percorso verso il vicino è migliore
            if neighbor_bytes not in g_cost or tentative_g_cost < g_cost[neighbor_bytes]:
                g_cost[neighbor_bytes] = tentative_g_cost
                priority = tentative_g_cost + manhattan_distance(neighbor_state)
                heappush(frontier, (priority, neighbor_bytes))
                came_from[neighbor_bytes] = (current_bytes, action)

    raise ValueError("Non è stato trovato alcun percorso valido.")

def reconstruct_path(came_from, current_bytes):
    """Ricostruisce il percorso dalla soluzione allo stato iniziale."""
    path = []
    while current_bytes in came_from:
        prev_bytes, action = came_from[current_bytes]
        path.append(action)
        current_bytes = prev_bytes
    return path[::-1]  # Inverti per ottenere il percorso dall'inizio alla fine

In [5]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[3, 6, 0],
       [5, 2, 7],
       [8, 1, 4]])

In [6]:
# Risoluzione del puzzle con A*
solution = a_star(state)

# Applica la soluzione passo dopo passo
current_state = state.copy()
print("\nRisoluzione passo-passo:")
for step, action in enumerate(solution, start=1):
    current_state = do_action(current_state, action)
    print(f"Passo {step}:")
    print(current_state)

print("\nPuzzle risolto!")


Risoluzione passo-passo:
Passo 1:
[[3 0 6]
 [5 2 7]
 [8 1 4]]
Passo 2:
[[3 2 6]
 [5 0 7]
 [8 1 4]]
Passo 3:
[[3 2 6]
 [5 1 7]
 [8 0 4]]
Passo 4:
[[3 2 6]
 [5 1 7]
 [0 8 4]]
Passo 5:
[[3 2 6]
 [0 1 7]
 [5 8 4]]
Passo 6:
[[3 2 6]
 [1 0 7]
 [5 8 4]]
Passo 7:
[[3 2 6]
 [1 8 7]
 [5 0 4]]
Passo 8:
[[3 2 6]
 [1 8 7]
 [5 4 0]]
Passo 9:
[[3 2 6]
 [1 8 0]
 [5 4 7]]
Passo 10:
[[3 2 6]
 [1 0 8]
 [5 4 7]]
Passo 11:
[[3 0 6]
 [1 2 8]
 [5 4 7]]
Passo 12:
[[0 3 6]
 [1 2 8]
 [5 4 7]]
Passo 13:
[[1 3 6]
 [0 2 8]
 [5 4 7]]
Passo 14:
[[1 3 6]
 [5 2 8]
 [0 4 7]]
Passo 15:
[[1 3 6]
 [5 2 8]
 [4 0 7]]
Passo 16:
[[1 3 6]
 [5 2 8]
 [4 7 0]]
Passo 17:
[[1 3 6]
 [5 2 0]
 [4 7 8]]
Passo 18:
[[1 3 0]
 [5 2 6]
 [4 7 8]]
Passo 19:
[[1 0 3]
 [5 2 6]
 [4 7 8]]
Passo 20:
[[1 2 3]
 [5 0 6]
 [4 7 8]]
Passo 21:
[[1 2 3]
 [0 5 6]
 [4 7 8]]
Passo 22:
[[1 2 3]
 [4 5 6]
 [0 7 8]]
Passo 23:
[[1 2 3]
 [4 5 6]
 [7 0 8]]
Passo 24:
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Puzzle risolto!
